In [1]:
import sys, win32com.client
import pandas as pd
import numpy as np
import sapscript as ss

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.mode.chained_assignment = None

In [12]:
def replace_minus(string):
    string = string.replace('.','')
    string = string.replace(' ','')
    if string[-1] == '-':
        string = -int(string.replace('-',''))
    else:
        string = int(string)
    return(string)




def read_zam(session, group_number):
    "Читаем группу замещающих товаров"
    zam = []
    #заходим в группы замещ.товаров с номером group_number
    session.StartTransaction(Transaction="zgrzt3")
    session.findById("wnd[0]/usr/ctxtS_DPGID-LOW").text = group_number
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    
    #в верхней ALV проваливаемся в группу
    grid1 = session.findById("wnd[0]/usr/cntlALV_0100/shellcont/shellcont/shell/shellcont[0]/shell")
    grid1.clickCurrentCell()
    
    #в нижней ALV копируем товары и их приоритеты в отдельный датафрейм
    grid2 = session.findById("wnd[0]/usr/cntlALV_0100/shellcont/shellcont/shell/shellcont[1]/shell")
    rows = grid2.rowcount
    
    for row in range(0,rows):
        matnr = grid2.getcellvalue(row,"MATNR")
        prior = int(grid2.getcellvalue(row,"DPGPR"))
        segment = grid2.getcellvalue(row, "MSTAE_" + company_code)
        
        zam.append({'Тов':matnr,
                    'Пр': prior,
                    'ЖЦ': segment})
    dataframe = pd.DataFrame(zam,
                             columns = ['Тов',
                                        'Пр',
                                        'ЖЦ'])
    return dataframe





def read_stock(session, plants, materials):
    """Читаем запасы на 0010 складе в ZMB52"""
    stock = []
    session.StartTransaction(Transaction="zmb52")
    session.findById("wnd[0]/usr/btn%_SO_WERKS_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[16]").press()
    #копируем заводы и товары в буфер
    werks = pd.DataFrame(plants)
    werks.to_clipboard(index=False, header=False, line_terminator='\r\n')
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    session.findById("wnd[0]/usr/ctxtSO_LGORT-LOW").text = "0010"
    
    session.findById("wnd[0]/usr/btn%_SO_MATNR_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[16]").press()
    
    #копируем товары и товары в буфер
    materials.to_clipboard(index=False, header=False, line_terminator='\r\n')
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    
    #Ставим флаги резервов
    session.findById("wnd[0]/usr/chkP_REZERV").selected = 0
    session.findById("wnd[0]/usr/chkP_IMDCQ").selected = 0
    
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    
    grid = session.findById("wnd[0]/shellcont/shell")
    rows = grid.rowcount

    #делаем прокрутку, если много рядов
    if rows > max_rows:
        visible_row = max_rows + 1
        while visible_row < rows:
            grid.firstVisibleRow = visible_row
            visible_row = visible_row + max_rows
            
    for row in range(0,rows):
        werk = grid.getcellvalue(row,"WERKS")
        matnr = grid.getcellvalue(row,"MATNR")
        qty = replace_minus(grid.getcellvalue(row,"QTY"))
        if werk == '':
            continue
        stock.append({'Звд':werk,
                      'Тов': matnr,
                      'Запас': qty})
        
    dataframe = pd.DataFrame(stock,
                             columns = ['Звд',
                                        'Тов',
                                        'Запас'])
    return dataframe





def count_inact_stock(zam, stock):
    """Считаем количество неактивных товаров с запасом на магазине"""
    #Размножаем все замещающие товары по количеству заводов и добавляем к ним запасы.
    zam_with_plant = pd.DataFrame()
    for plant in plants:
        zam['Звд'] = plant
        zam_with_plant = zam_with_plant.append(zam)
    zam = zam.drop(columns='Звд')
    zam_with_stock = zam_with_plant.merge(stock, on=['Тов','Звд'], how='outer')
    zam_with_stock.fillna(0, inplace=True)
    zam_with_stock['Запас'] = zam_with_stock['Запас'].astype(int)
    cols = ['Звд', 'Тов','ЖЦ','Пр','Запас']
    zam_with_stock = zam_with_stock[cols]
    
    #Находим количество неактивных товаров с запасом на каждом из заводов
    inact_stock = zam_with_stock[(zam_with_stock['ЖЦ'].isin(inactive_segment)) & (zam_with_stock['Запас']>0)].groupby('Звд').count()['Запас']
    #Если вдруг попались заводы без запасов вообще, то добавляем их в список с нулем
    for plant in plants:
        if plant not in inact_stock.index:
            inact_stock = inact_stock.append(pd.Series([0],index = [plant]))
    return inact_stock




def read_az(session, materials):
    """Читаем данные из проектов автозаказа"""
    #заходим в позиции проектов
    session.StartTransaction(Transaction="zreplen03")
    session.findById("wnd[0]/usr/ctxtP_DATE").text = date
    session.findById("wnd[0]/usr/ctxtS_DATA0-LOW").text = ""
    
    #Вставляем заводы из буфера
    session.findById("wnd[0]/usr/btn%_S_EMWRK_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[16]").press()
    #копируем заводы в буфер
    werks = pd.DataFrame(plants)
    werks.to_clipboard(index=False, header=False, line_terminator='\r\n')
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    
    #Вставляем список товаров из буфера
    session.findById("wnd[0]/usr/btn%_S_MATNR_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[16]").press()
    #копируем товары в буфер
    materials.to_clipboard(index=False, header=False, line_terminator='\r\n')
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    
    
    session.findById("wnd[0]/usr/ctxtS_SDWRK-LOW").text = vendor
    session.findById("wnd[0]/usr/radVIEW_POS").select()
    session.findById("wnd[0]/usr/ctxtP_VARI").text = variant
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    
    #кол-во строк в ALV
    grid = session.findById("wnd[0]/usr/cntlALV0101/shellcont/shell")
    rows = grid.rowcount
    
    #делаем прокрутку, если много рядов
    if rows > max_rows:
        visible_row = max_rows + 1
        while visible_row < rows:
            grid.firstVisibleRow = visible_row
            visible_row = visible_row + max_rows
            
            
    data = []
    for row in range(0,rows):
        matnr = grid.getcellvalue(row,"MATNR")            #товар
        emwrk = grid.getcellvalue(row,"EMWRK")            #завод
        norm  = int(grid.getcellvalue(row,"DPGCN"))           #норматив для подформата
        prior = int(grid.getcellvalue(row,"DPGPR"))       #приоритет в группе замещ.товаров
        null_az = grid.getcellvalue(row,"DPGCL")          #обнуление позиции по зам.товара в АЗ
        mag_instock = replace_minus(grid.getcellvalue(row,"TRAME_RI"))  #запас в пути на магазин
        mag_stock = replace_minus(grid.getcellvalue(row,"LABST_R_0"))+mag_instock   #запас магазина
        m_az = grid.getcellvalue(row,"DPGWS")    #Число М - неприоритетные товары с запасом
        zakaz = grid.getcellvalue(row,"MENGE_NE2")    #Заказ, ШТ
        salerate = grid.getcellvalue(row,"AQUAN")    #Скорость продаж
        
        data.append({'Звд'    :emwrk,
                     'Тов'  : matnr,
                     'Нрм'   :norm,
                     'Маг_з'  :mag_stock,
                     'ПрА'  :prior,
                     'M_АЗ'   :m_az,
                     'Обн_АЗ' :null_az,
                     'ШТ'     :zakaz,
                     'Скр'    :salerate})
        
        dataframe = pd.DataFrame(data,
                             columns = ['Звд',
                                        'Тов',
                                        'Нрм',
                                        'Маг_з',
                                        'ПрА',
                                        'M_АЗ',
                                        'Обн_АЗ',
                                        'ШТ',
                                        'Скр'])
    return dataframe





def count_N(az, inact_stock):
    """Вычисляем N = max(0, норматив - неактив с запасом)"""
    inact_stock_df = inact_stock.reset_index()
    inact_stock_df.columns = ['Звд','Inact']
    azN = az.merge(inact_stock_df)
    azN['N'] = azN['Нрм'] - azN['Inact']
    azN['N'] = azN['N'].apply(lambda x: int(max(0,x)))
    return azN




def analize(azN, zam, stock):
    """Cовмещаем данные из группы замещ.товаров и автозаказа и ставим флаги обнуления"""
    final_data = pd.merge(azN, zam, on='Тов')
    cols = ['Звд',
            'Тов',
            'Пр',
            'ПрА',
            'ЖЦ',
            'Нрм',
            'Маг_з',
            'Обн1',
            'Обн2',
            'N',
            'Inact',
            'M',
            'M_АЗ',
            'Обн_ит',
            'Обн_АЗ',
            'ШТ',
            'Скр']
    final_data = final_data.sort_values(by=['Звд','Пр'])
    final_data['Обн1'] = ''
    final_data['Обн2'] = ''
    final_data['M'] = ''
    final_data['Обн_ит'] = ''
    final_data = final_data[cols]
    
    dataframe = pd.DataFrame()
    for plant in plants:
        final_plant_data = final_data[final_data['Звд'] == plant]
        if len(final_plant_data)==0:
            continue
        N = int(final_plant_data['N'].iloc[0])  #вычислили N раньше для каждого завода
        
        #Обнуляем позиции, не попавшие в первые N 
        final_plant_data.loc[final_plant_data.iloc[N:].index,'Обн1'] = 'X'

        #Вычисляем M - кол-в товаров с запасом и флагом обнуления
        M = len(final_plant_data.loc[(final_plant_data['Маг_з']>0)&(final_plant_data['Обн1']=='X')])
        
        #Вычисляем M1 - количество товаров с активным ЖЦ на запасах магазина, но не в проектах.
        #Товары на запасах, но не в проектах заказов
        stock_outproject = stock[(stock['Звд']==plant)&(~stock['Тов'].isin(azN[azN['Звд']==plant]['Тов']))]
        stock_outproject = stock_outproject.merge(zam) 
        M1 = len(stock_outproject[(stock_outproject['Запас']>0) & (~stock_outproject['ЖЦ'].isin(inactive_segment))])
        
        #Прибавляем M1 к M, увеличивая количество обнуляемых товаров в проекте, 
        #т.к. часть замещающих товаров есть на запасах магазина и больше везти туда не надо
        M = M + M1
        
        final_plant_data['M'] = M
        
        if M != 0:
            final_plant_data.loc[(final_plant_data[final_plant_data['Обн1']!='X'].index[-M:]),['Обн2']]='X'
        final_plant_data.loc[((final_plant_data['Обн1']=='X') | (final_plant_data['Обн2']=='X')), ['Обн_ит']]='X'
        final_plant_data = final_plant_data.reset_index(drop=True)
        dataframe = dataframe.append(final_plant_data)
    return dataframe


#-Sub Main-------------------------------------------------------------- 
def Main():
    try:
        SapGuiAuto = win32com.client.GetObject("SAPGUI")
        if not type(SapGuiAuto) == win32com.client.CDispatch:
            return
        application = SapGuiAuto.GetScriptingEngine
        if not type(application) == win32com.client.CDispatch:
            SapGuiAuto = None
            return
        connection = application.Children(0)
        if not type(connection) == win32com.client.CDispatch:
            application = None
            SapGuiAuto = None
            return
        session = connection.Children(0)
        if not type(session) == win32com.client.CDispatch:
            connection = None
            application = None
            SapGuiAuto = None
            return
        
        #читаем группу замещающих товаров
        zam = read_zam(session, group_number)
        materials = pd.DataFrame(zam['Тов'])
        stock = read_stock(session, plants, materials)
        inact_stock = count_inact_stock(zam, stock)
        zam = zam.drop(columns='Звд')
        az = read_az(session, materials)
        azN = count_N(az, inact_stock)
        final_data = analize(azN,zam, stock)
        
        return zam, stock, inact_stock, az, azN, final_data
    
    except:
        print('Ошибка')
        print(sys.exc_info())

    finally:
        session = None
        connection = None
        application = None
        SapGuiAuto = None      
#-Main------------------------------------------------------------------

def total():
    """Выводим весь расчет"""
    for plant in plants:
        final_plant_data = final_data[final_data['Звд']==plant]
        if len(final_plant_data)>0:
            print(final_plant_data, end='\n\n')

def delta():
    """Выводим строки с отличающимися флагами обнуления"""
    delta = final_data[((final_data['Обн_ит']=='X') & (final_data['Обн_АЗ']!='X'))|((final_data['Обн_ит']!='X') & (final_data['Обн_АЗ']=='X'))]
    for plant in delta['Звд'].unique():
        final_plant_data = final_data[final_data['Звд']==plant]
        if len(final_plant_data)>0:
            print(final_plant_data, end='\n\n')
    

def test_plant(test_plant):
    """Выводим детальную информацию по отдельному заводу"""
    #Товары на запасах, но не в проектах заказов
    stock_outproject = stock[(stock['Звд']==test_plant)&(~stock['Тов'].isin(azN[azN['Звд']==test_plant]['Тов']))]
    stock_outproject = stock_outproject.merge(zam)
    #количество товаров с активным ЖЦ на запасах магазина, но не в проектах.
    #stock_outproject[(stock_outproject['Запас']>0) & (~stock_outproject['ЖЦ'].isin(inactive_segment))]
    print('Товары, не вошедшие в проект, но с запасами на магазине:')
    print(stock_outproject[(stock_outproject['Запас']>0)], end='\n\n')
    print('Строки проекта заказа:')
    print(final_data[final_data['Звд']==test_plant])

## ERD300 ##

In [189]:
company_code = "1000"    #код компании
inactive_segment = ['Z1','Z4','Z5']  #неактивные сегменты ЖЦ
group_number = 61      #номер группы замещающих товаров
date = "19.07.2019"    #дата для чтения проектов автозаказа
#plants = ["2010","2011","2013","2014","2015","2016","2017","2018","2019","2020","2059"]        #номер завода
plants = ["2010","2011","2013","2014","2015","2016","2017","2018","2019","2020","2059"]        #номер завода

vendor = '100000000'  #код внешнего поставщика
variant = "/all"
max_rows = 30          #максимум строк на экране ALV, выше которого нужно делать page down

if __name__ == "__main__":
    zam, stock, inact_stock, az, azN, final_data = Main()
    
for plant in plants:
    final_plant_data = final_data[final_data['Звд']==plant]
    if len(final_plant_data)>0:
        print(final_plant_data, end='\n\n')

Ошибка
(<class 'pywintypes.com_error'>, com_error(-2147352567, 'Ошибка.', (619, 'SAP Frontend Server', 'The control could not be found by id.', 'C:\\Program Files (x86)\\SAP\\FrontEnd\\SAPgui\\sapfront.HLP', 393215, 0), None), <traceback object at 0x000000366DC01908>)


TypeError: 'NoneType' object is not iterable

## ERI ##

In [13]:
company_code     = "1000"            #код компании
inactive_segment = ['Z1','Z4','Z5']  #неактивные сегменты ЖЦ
group_number     = 14                #номер группы замещающих товаров
date             = "25.07.2019"      #дата для чтения проектов автозаказа
vendor           = '100000028'       #код внешнего поставщика
variant          = "/all"            #вариант запуска zreplen03
max_rows         = 30                #максимум строк на экране ALV, выше которого нужно делать page down

plants = ["2010","2013","2014","2015","2016","2018","2020","2021","2022","2023","2025"]
#"2027","2028","2029","2030","2031","2032","2033","2034","2035","2036","2037",
#"2038","2039","2040","2041","2043","2044","2046","2049","2050","2051","2052"]
#"2053","2054","2055","2056","2057","2058","2059","2060","2061","2062","2063",
#"2064","2065","2066","2068","2069","2070","2071","2075","2076","2077","2078",
#"2080","2082","2083","2084","2085","2086","2088","2089","2090","2091","2092",
#"2093","2094","2095","2097","2098","2099","2100","2101","2102","2103","2104",
#"2105","2106","2107","2108","2109","2111","2112","2113","2114","2115","2116",
#"2117","2118","2119","2120","2121","2122","2124","2125","2126","2127","2128",
#"2129","2130","2131","2132","2133","2134","2136","2137","2140","2141","2142",
#"2143","2144","2145","2146","2147","2149","2151","2152","2153","2154","2155",
#"2156","2157","2158","2159","2160","2161","2162","2163","2164","2165","2166"]

zam, stock, inact_stock, az, azN, final_data = Main()
    
#for plant in plants:
#    final_plant_data = final_data[final_data['Завод']==plant]
#    if len(final_plant_data)>0:
#        print(final_plant_data, end='\n\n')

In [17]:
test_plant('2023')

Товары, не вошедшие в проект, но с запасами на магазине:
    Звд         Тов  Запас  Пр  ЖЦ
0  2023  1000015466      3  27  Z1
1  2023  1000021514      1  29  Z4
2  2023  1000021515      2  30  Z2
3  2023  1000045856      2  38  Z1

Строки проекта заказа:
    Звд         Тов  Пр  ПрА  ЖЦ  Нрм  Маг_з Обн1 Обн2  N  Inact  M M_АЗ Обн_ит Обн_АЗ ШТ    Скр
0  2023       30600   4    4  Z2    5      1         X  2      3  7    7      X      X  0      0
1  2023   101929100  13   13  Z2    5      2         X  2      3  7    7      X      X  0      0
2  2023  1000005789  26   26  Z2    5      1    X       2      3  7    7      X      X  0      0
3  2023  1000015467  28   28  Z2    5      2    X       2      3  7    7      X      X  0      0
4  2023  1000021518  31   31  Z2    5      2    X       2      3  7    7      X      X  0      0
5  2023  1000021520  32   32  Z2    5      2    X       2      3  7    7      X      X  0      0
6  2023  1000074921  46   46  Z2    5      3    X       2      3 

In [288]:
test_plant('2018')

Товары, не вошедшие в проект, но с запасами на магазине:
    Звд         Тов  Запас  Пр  ЖЦ
0  2018  1000015466      2  27  Z1
1  2018  1000021515      2  30  Z2
2  2018  1000045856      2  38  Z1

Строки проекта заказа:
    Звд         Тов  Пр  ПрА  ЖЦ  Нрм  Маг_з Обн1 Обн2  N  Inact  M M_АЗ Обн_ит Обн_АЗ ШТ    Скр
0  2018       30600   4    4  Z2    7      3            5      2  4    4             X  0      0
1  2018   101929100  13   13  Z2    7      1         X  5      2  4    4      X      X  0      0
2  2018  1000005789  26   26  Z2    7      2         X  5      2  4    4      X      X  0      0
3  2018  1000015467  28   28  Z2    7      3         X  5      2  4    4      X      X  0      0
4  2018  1000021518  31   31  Z2    7      1         X  5      2  4    4      X      X  0  0,024
5  2018  1000021520  32   32  Z2    7      2    X       5      2  4    4      X      X  0  0,024
6  2018  1000074921  46   46  Z2    7      1    X       5      2  4    4      X      X  0  0,024
7  

In [15]:
delta()

    Звд         Тов  Пр  ПрА  ЖЦ  Нрм  Маг_з Обн1 Обн2  N  Inact  M M_АЗ Обн_ит Обн_АЗ ШТ    Скр
0  2018       30600   4    4  Z2    7      3            5      2  4    4             X  0      0
1  2018   101929100  13   13  Z2    7      1         X  5      2  4    4      X      X  0      0
2  2018  1000005789  26   26  Z2    7      2         X  5      2  4    4      X      X  0      0
3  2018  1000015467  28   28  Z2    7      3         X  5      2  4    4      X      X  0      0
4  2018  1000021518  31   31  Z2    7      1         X  5      2  4    4      X      X  0  0,024
5  2018  1000021520  32   32  Z2    7      2    X       5      2  4    4      X      X  0  0,024
6  2018  1000074921  46   46  Z2    7      1    X       5      2  4    4      X      X  0  0,024
7  2018  1000074922  47   47  Z2    7      2    X       5      2  4    4      X      X  0      0

    Звд         Тов  Пр  ПрА  ЖЦ  Нрм  Маг_з Обн1 Обн2  N  Inact  M M_АЗ Обн_ит Обн_АЗ ШТ    Скр
0  2021       30600   4    4 

    Звд         Тов  Пр  Пр_АЗ  ЖЦ  Нрм  Маг_з Обн1 Обн2  N  Inact  M M_АЗ Обн_ит Обн_АЗ ШТ    Скр
0  2010       30600   4      4  Z2    7      1         X  4      3  5    5      X      X  0      0
1  2010   101929100  13     13  Z2    7      1         X  4      3  5    5      X      X  0      0
2  2010  1000005789  26     26  Z2    7      2         X  4      3  5    5      X      X  0      0
3  2010  1000015467  28     28  Z2    7      2         X  4      3  5    5      X      X  0      0
4  2010  1000021518  31     31  Z2    7      3    X       4      3  5    5      X      X  0      0
5  2010  1000021520  32     32  Z2    7      2    X       4      3  5    5      X      X  0      0
6  2010  1000074921  46     46  Z2    7      2    X       4      3  5    5      X      X  0      0
7  2010  1000074922  47     47  Z2    7      2    X       4      3  5    5      X      X  0  0,024

    Звд         Тов  Пр  Пр_АЗ  ЖЦ  Нрм  Маг_з Обн1 Обн2  N  Inact  M M_АЗ Обн_ит Обн_АЗ ШТ    Скр
0  2013  

7  2035  1000074922  47     47  Z2    7      2    X       4      3  4    4      X      X  0   0

    Звд         Тов  Пр  Пр_АЗ  ЖЦ  Нрм  Маг_з Обн1 Обн2  N  Inact  M M_АЗ Обн_ит Обн_АЗ ШТ    Скр
0  2036       30600   4      4  Z2    5      3         X  2      3  8    8      X      X  0  0,024
1  2036   101929100  13     13  Z2    5      1         X  2      3  8    8      X      X  0      0
2  2036  1000005789  26     26  Z2    5      2    X       2      3  8    8      X      X  0      0
3  2036  1000015467  28     28  Z2    5      1    X       2      3  8    8      X      X  0  0,024
4  2036  1000021518  31     31  Z2    5      2    X       2      3  8    8      X      X  0      0
5  2036  1000021520  32     32  Z2    5      4    X       2      3  8    8      X      X  0      0
6  2036  1000074921  46     46  Z2    5      2    X       2      3  8    8      X      X  0      0
7  2036  1000074922  47     47  Z2    5      2    X       2      3  8    8      X      X  0      0

    Звд    